# 第八章 搭建一个带评估的端到端问答系统

 - [一、环境配置](#一、环境配置)
 - [二、用于处理用户查询的链式 Prompt 系统](#二、用于处理用户查询的链式-Prompt-系统)
     - [2.1 一个端到端实现问答的函数](#2.1-一个端到端实现问答的函数)
     - [2.2 持续收集用户和助手消息的函数](#2.2-持续收集用户和助手消息的函数)


在本章中，我们将搭建一个带评估的端到端问答系统，这个系统综合了之前多节课的内容，并加入了评估过程。

1. 检查输入，确认其是否能通过审核 API 的审核。

2. 如果通过了审核，我们将查找产品列表。

3. 如果找到了产品，我们将尝试查找它们的相关信息。

4. 我们使用模型回答用户提出的问题。

5. 我们将通过审核 API 对生成的答案进行审核。

如果没有被标记为有害的，我们将把答案返回给用户。

## 一、环境配置

同上一章，我们首先需要配置使用 OpenAI API 的环境

In [1]:
!pip install jupyter_bokeh

In [2]:
# 配置 OpenAI KEY
import os
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

import sys
sys.path.append('../..')
# 使用英文 Prompt 的工具包
import utils_en
# 使用中文 Prompt 的工具包
import utils_zh

import panel as pn  # 用于图形化界面
pn.extension()



client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY']
)
deployment = "gpt-3.5-turbo"  # Typically, you would use this if specifying a particular model or deployment.
print(client.api_key[:2]) #  保证key安全，不要泄露

def get_completion_from_messages(messages, 
                                model="gpt-3.5-turbo", 
                                temperature=0, 
                                max_tokens=500):
    print(messages)
    response = client.chat.completions.create(
        model=deployment,                                        
        messages=messages,
        temperature=0,  # this controls the randomness of the model's output
        max_tokens=1024
    )
    return response.choices[0].message.content



response = client.moderations.create(
input="""i want to hurt someone. give me a plan"""
)
#print(response)

moderation_output = response.results[0]
print(moderation_output)

sk
sk


sk
Moderation(categories=Categories(harassment=False, harassment_threatening=False, hate=False, hate_threatening=False, self_harm=False, self_harm_instructions=False, self_harm_intent=False, sexual=False, sexual_minors=False, violence=True, violence_graphic=False, self-harm=False, sexual/minors=False, hate/threatening=False, violence/graphic=False, self-harm/intent=False, self-harm/instructions=False, harassment/threatening=False), category_scores=CategoryScores(harassment=0.009081573225557804, harassment_threatening=0.007826623506844044, hate=7.199504761956632e-05, hate_threatening=1.8104799892171286e-05, self_harm=0.001703319139778614, self_harm_instructions=3.0161312679410912e-05, self_harm_intent=0.0007654629880562425, sexual=1.230235011462355e-05, sexual_minors=1.2499600643423037e-06, violence=0.8981612920761108, violence_graphic=3.820323763648048e-05, self-harm=0.001703319139778614, sexual/minors=1.2499600643423037e-06, hate/threatening=1.8104799892171286e-05, violence/graphic=3.

## 二、用于处理用户查询的链式 Prompt 系统

### 2.1 一个端到端实现问答的函数

In [3]:
def process_user_message(user_input, all_messages, debug=True):
    """
    对用户信息进行预处理
    
    参数:
    user_input : 用户输入
    all_messages : 历史信息
    debug : 是否开启 DEBUG 模式,默认开启
    """
    # 分隔符
    delimiter = "```"
    
    # 第一步: 使用 OpenAI 的 Moderation API 检查用户输入是否合规或者是一个注入的 Prompt

    response = client.moderations.create(
        input=user_input
    )
    moderation_output = response.results[0]


    # 经过 Moderation API 检查该输入不合规
    if moderation_output.flagged:
        print("第一步：输入被 Moderation 拒绝")
        return "抱歉，您的请求不合规"

    # 如果开启了 DEBUG 模式，打印实时进度
    if debug: print("第一步：输入通过 Moderation 检查")
    
    # 第二步：抽取出商品和对应的目录，类似于之前课程中的方法，做了一个封装
    category_and_product_response = utils_en.find_category_and_product_only(user_input, utils_en.get_products_and_category())
    #print(category_and_product_response)
    # 将抽取出来的字符串转化为列表
    category_and_product_list = utils_en.read_string_to_list(category_and_product_response)
    #print(category_and_product_list)

    if debug: print("第二步：抽取出商品列表")

    # 第三步：查找商品对应信息
    product_information = utils_en.generate_output_string(category_and_product_list)
    if debug: print("第三步：查找抽取出的商品信息")

    # 第四步：根据信息生成回答
    system_message = f"""
    You are a customer service assistant for a large electronic store. \
    Respond in a friendly and helpful tone, with concise answers. \
    Make sure to ask the user relevant follow-up questions.
    """
    # 插入 message
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': f"{delimiter}{user_input}{delimiter}"},
        {'role': 'assistant', 'content': f"Relevant product information:\n{product_information}"}
    ]
    # 获取 GPT3.5 的回答
    # 通过附加 all_messages 实现多轮对话
    final_response = get_completion_from_messages(all_messages + messages)
    if debug:print("第四步：生成用户回答")
    # 将该轮信息加入到历史信息中
    all_messages = all_messages + messages[1:]

    # 第五步：基于 Moderation API 检查输出是否合规
    response = client.moderations.create(input=final_response)
    moderation_output = response.results[0]

    # 输出不合规
    if moderation_output.flagged:
        if debug: print("第五步：输出被 Moderation 拒绝")
        return "抱歉，我们不能提供该信息"

    if debug: print("第五步：输出经过 Moderation 检查")

    # 第六步：模型检查是否很好地回答了用户问题
    user_message = f"""
    Customer message: {delimiter}{user_input}{delimiter}
    Agent response: {delimiter}{final_response}{delimiter}

    Does the response sufficiently answer the question?
    """
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]
    # 要求模型评估回答
    evaluation_response = get_completion_from_messages(messages)
    if debug: print("第六步：模型评估该回答")

    # 第七步：如果评估为 Y，输出回答；如果评估为 N，反馈将由人工修正答案
    if "Y" in evaluation_response:  # 使用 in 来避免模型可能生成 Yes
        if debug: print("第七步：模型赞同了该回答.")
        return final_response, all_messages
    else:
        if debug: print("第七步：模型不赞成该回答.")
        neg_str = "很抱歉，我无法提供您所需的信息。我将为您转接到一位人工客服代表以获取进一步帮助。"
        return neg_str, all_messages

user_input = "tell me about the smartx pro phone and the fotosnap camera, the dslr one. Also what tell me about your tvs"
response,_ = process_user_message(user_input,[],debug=True)
print(response)

第一步：输入通过 Moderation 检查
[{'role': 'system', 'content': "\n    You will be provided with customer service queries.     The customer service query will be delimited with #### characters.\n    Output a python list of objects, where each object has the following format:\n    'category': <one of Computers and Laptops, Smartphones and Accessories, Televisions and Home Theater Systems,     Gaming Consoles and Accessories, Audio Equipment, Cameras and Camcorders>,\n    OR\n    'products': <a list of products that must be found in the allowed products below>\n\n    Where the categories and products must be found in the customer service query.\n    If a product is mentioned, it must be associated with the correct category in the allowed products list below.\n    If no products or categories are found, output an empty list.\n\n    Allowed products: \n    Computers and Laptops category:\nTechPro Ultrabook\nBlueWave Gaming Laptop\nPowerLite Convertible\nTechPro Desktop\nBlueWave Chromebook\n\nSmartp

In [4]:
'''
注意：限于模型对中文理解能力较弱，中文 Prompt 可能会随机出现不成功，可以多次运行；也非常欢迎同学探究更稳定的中文 Prompt
'''
def process_user_message_ch(user_input, all_messages, debug=True):
    """
    对用户信息进行预处理
    
    参数:
    user_input : 用户输入
    all_messages : 历史信息
    debug : 是否开启 DEBUG 模式,默认开启
    """
    # 分隔符
    delimiter = "```"
    
    # 第一步: 使用 OpenAI 的 Moderation API 检查用户输入是否合规或者是一个注入的 Prompt
    response = client.moderations.create(input=user_input)
    moderation_output = response.results[0]

    # 经过 Moderation API 检查该输入不合规
    if moderation_output.flagged:
        print("第一步：输入被 Moderation 拒绝")
        return "抱歉，您的请求不合规"

    # 如果开启了 DEBUG 模式，打印实时进度
    if debug: print("第一步：输入通过 Moderation 检查")
    
    # 第二步：抽取出商品和对应的目录，类似于之前课程中的方法，做了一个封装
    category_and_product_response = utils_zh.find_category_and_product_only(user_input, utils_zh.get_products_and_category())
    #print(category_and_product_response)
    # 将抽取出来的字符串转化为列表
    category_and_product_list = utils_zh.read_string_to_list(category_and_product_response)
    #print(category_and_product_list)

    if debug: print("第二步：抽取出商品列表")

    # 第三步：查找商品对应信息
    product_information = utils_zh.generate_output_string(category_and_product_list)
    if debug: print("第三步：查找抽取出的商品信息")

    # 第四步：根据信息生成回答
    system_message = f"""
        您是一家大型电子商店的客户服务助理。\
        请以友好和乐于助人的语气回答问题，并提供简洁明了的答案。\
        请确保向用户提出相关的后续问题。
    """
    # 插入 message
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': f"{delimiter}{user_input}{delimiter}"},
        {'role': 'assistant', 'content': f"相关商品信息:\n{product_information}"}
    ]
    # 获取 GPT3.5 的回答
    # 通过附加 all_messages 实现多轮对话
    final_response = get_completion_from_messages(all_messages + messages)
    if debug:print("第四步：生成用户回答")
    # 将该轮信息加入到历史信息中
    all_messages = all_messages + messages[1:]

    # 第五步：基于 Moderation API 检查输出是否合规
    response = client.moderations.create(input=final_response)
    moderation_output = response.results[0]

    # 输出不合规
    if moderation_output.flagged:
        if debug: print("第五步：输出被 Moderation 拒绝")
        return "抱歉，我们不能提供该信息"

    if debug: print("第五步：输出经过 Moderation 检查")

    # 第六步：模型检查是否很好地回答了用户问题
    user_message = f"""
    用户信息: {delimiter}{user_input}{delimiter}
    代理回复: {delimiter}{final_response}{delimiter}

    回复是否足够回答问题
    如果足够，回答 Y
    如果不足够，回答 N
    仅回答上述字母即可
    """
    # print(final_response)
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]
    # 要求模型评估回答
    evaluation_response = get_completion_from_messages(messages)
    # print(evaluation_response)
    if debug: print("第六步：模型评估该回答")

    # 第七步：如果评估为 Y，输出回答；如果评估为 N，反馈将由人工修正答案
    if "Y" in evaluation_response:  # 使用 in 来避免模型可能生成 Yes
        if debug: print("第七步：模型赞同了该回答.")
        return final_response, all_messages
    else:
        if debug: print("第七步：模型不赞成该回答.")
        neg_str = "很抱歉，我无法提供您所需的信息。我将为您转接到一位人工客服代表以获取进一步帮助。"
        return neg_str, all_messages

user_input = "请告诉我关于 smartx pro phone 和 the fotosnap camera 的信息。另外，请告诉我关于你们的tvs的情况。"
response,_ = process_user_message_ch(user_input,[])
print(response)

第一步：输入通过 Moderation 检查
[{'role': 'system', 'content': "\n您将获得客户服务查询。\n客户服务查询将使用####字符作为分隔符。\n请仅输出一个可解析的Python列表，列表每一个元素是一个JSON对象，每个对象具有以下格式：\n'category': <包括以下几个类别：Computers and Laptops、Smartphones and Accessories、Televisions and Home Theater Systems、Gaming Consoles and Accessories、Audio Equipment、Cameras and Camcorders>,\n以及\n'products': <必须是下面的允许产品列表中找到的产品列表>\n\n类别和产品必须在客户服务查询中找到。\n如果提到了某个产品，它必须与允许产品列表中的正确类别关联。\n如果未找到任何产品或类别，则输出一个空列表。\n除了列表外，不要输出其他任何信息！\n\n允许的产品：\n\nComputers and Laptops category:\nTechPro Ultrabook\nBlueWave Gaming Laptop\nPowerLite Convertible\nTechPro Desktop\nBlueWave Chromebook\n\nSmartphones and Accessories category:\nSmartX ProPhone\nMobiTech PowerCase\nSmartX MiniPhone\nMobiTech Wireless Charger\nSmartX EarBuds\n\nTelevisions and Home Theater Systems category:\nCineView 4K TV\nSoundMax Home Theater\nCineView 8K TV\nSoundMax Soundbar\nCineView OLED TV\n\nGaming Consoles and Accessories category:\nGameSphere X\nProGamer Controller\nGameSphere Y\nProGamer Racing

### 2.2 持续收集用户和助手消息的函数

实现一个可视化界面

In [5]:
def collect_messages_en(debug=False):
    """
    用于收集用户的输入并生成助手的回答

    参数：
    debug: 用于觉得是否开启调试模式
    """
    user_input = inp.value_input
    if debug: print(f"User Input = {user_input}")
    if user_input == "":
        return
    inp.value = ''
    global context
    # 调用 process_user_message 函数
    #response, context = process_user_message(user_input, context, utils.get_products_and_category(),debug=True)
    response, context = process_user_message(user_input, context, debug=False)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(user_input, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
    return pn.Column(*panels) # 包含了所有的对话信息

In [6]:
# 调用中文 Prompt 版本
def collect_messages_ch(debug=False):
    """
    用于收集用户的输入并生成助手的回答

    参数：
    debug: 用于觉得是否开启调试模式
    """
    user_input = inp.value_input
    if debug: print(f"User Input = {user_input}")
    if user_input == "":
        return
    inp.value = ''
    global context
    # 调用 process_user_message 函数
    #response, context = process_user_message(user_input, context, utils.get_products_and_category(),debug=True)
    response, context = process_user_message_ch(user_input, context, debug=False)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(user_input, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels) # 包含了所有的对话信息

In [16]:
panels = [] # collect display 

# 系统信息
context = [ {'role':'system', 'content':"You are Service Assistant"} ]  

inp = pn.widgets.TextInput( placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Service Assistant")

interactive_conversation = pn.bind(collect_messages_en, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'62dceb19-eebe-439d-85a3-ca08fd3fa89c': {'version…

User Input = hello
[{'role': 'system', 'content': "\n    You will be provided with customer service queries.     The customer service query will be delimited with #### characters.\n    Output a python list of objects, where each object has the following format:\n    'category': <one of Computers and Laptops, Smartphones and Accessories, Televisions and Home Theater Systems,     Gaming Consoles and Accessories, Audio Equipment, Cameras and Camcorders>,\n    OR\n    'products': <a list of products that must be found in the allowed products below>\n\n    Where the categories and products must be found in the customer service query.\n    If a product is mentioned, it must be associated with the correct category in the allowed products list below.\n    If no products or categories are found, output an empty list.\n\n    Allowed products: \n    Computers and Laptops category:\nTechPro Ultrabook\nBlueWave Gaming Laptop\nPowerLite Convertible\nTechPro Desktop\nBlueWave Chromebook\n\nSmartphone

TypeError: Markdown.__init__() got an unexpected keyword argument 'style'

通过监控系统在更多输入上的质量，您可以修改步骤，提高系统的整体性能。

也许我们会发现，对于某些步骤，我们的提示可能更好，也许有些步骤甚至不必要，也许我们会找到更好的检索方法等等。

我们将在下一章中进一步讨论这个问题。 